In [ ]:
! pip install -U backtrader

In [ ]:
# Strategy
from datetime import datetime
import backtrader

# Integrate Model
import sys
import warnings
import numpy
import pandas
warnings.filterwarnings('ignore')

In [ ]:
# Build Instance and draw single plot
#為了將GUI圖形顯示在 Jupyter notebook 
%matplotlib inline
from mpl_toolkits import mplot3d
from mpl_toolkits.mplot3d import Axes3D
import matplotlib
matplotlib.use('Agg')
import matplotlib.pyplot

In [ ]:
def Function_For_Build_Customize_Object(Parameter1, Parameter2):
    
    class Customize_Object(backtrader.Strategy):
        def __init__(self):
    
            self.data_open = self.datas[0].open
            self.data_high = self.datas[0].high
            self.data_low = self.datas[0].low
            self.data_close = self.datas[0].close
            self.data_volume = self.datas[0].volume

               
        def next(self):
                     
            print(pandas.DataFrame({"Open Price": self.data_open[0],
                                    "High Price": self.data_high[0],
                                    "Low Price": self.data_low[0],
                                    "Close Price": self.data_close[0],
                                    "Volumns": self.data_volume[0]},index=[0]))    
################################################################################################            
#                                      Space for Customize Strategy   
################################################################################################           

            if self.data_open[0] > 105:
                self.sell()
            if self.data_open[0] < 95:
                self.buy()
                
################################################################################################            
#                                      Space for Customize Strategy   
################################################################################################            

    return Customize_Object
            
    

In [ ]:
# 增加調用資料 函式庫yfinance
import yfinance
# 重寫類別使其格式可接受函式庫資料型態
class PandasData(backtrader.feed.DataBase):
    '''
    The ``dataname`` parameter inherited from ``feed.DataBase`` is the pandas
    DataFrame
    '''

    params = (
        # Possible values for datetime (must always be present)
        #  None : datetime is the "index" in the Pandas Dataframe
        #  -1 : autodetect position or case-wise equal name
        #  >= 0 : numeric index to the colum in the pandas dataframe
        #  string : column name (as index) in the pandas dataframe
        ('datetime', None),

        # Possible values below:
        #  None : column not present
        #  -1 : autodetect position or case-wise equal name
        #  >= 0 : numeric index to the colum in the pandas dataframe
        #  string : column name (as index) in the pandas dataframe
        ('open', -1),
        ('high', -1),
        ('low', -1),
        ('close', -1),
        ('volume', -1)
    )


In [ ]:
class Trading_Picture_Generator:


    # General
    Trading_Object_Name = ""
    Trading_Level_Multiplier = 1000
    
    Trading_DateTime_Start = datetime(2018,1,1)
    Trading_DateTime_End = datetime(2019,9,22)

    I_Am_Trading_Cerebro_Object = backtrader.Cerebro()
    
    # Parameter Performance Batch
    Parameter_Performance_Batch = []
    
    # Parameter for Customization Strategy
    Customization_Parameter1 = 10 
    Customization_Parameter2 = 20


    def __init__(
        self, 
        Trading_Object_Name_Input, 
        Trading_Level_Multiplier_Input, 
        Trading_DateTime_Start_Input, 
        Trading_DateTime_End_Input):

        self.Trading_Object_Name = Trading_Object_Name_Input
        self.Trading_Level_Multiplier = Trading_Level_Multiplier_Input

        self.Trading_DateTime_Start = Trading_DateTime_Start_Input
        self.Trading_DateTime_End = Trading_DateTime_End_Input


    def Function_RunTrading_Customize(self, Customization_Parameter1, Customization_Parameter2): 

        Cerebro_Object = backtrader.Cerebro()
        Cerebro_Object.broker.setcommission(mult=self.Trading_Level_Multiplier)
        Customize_StrategyObject = Function_For_Build_Customize_Object(Customization_Parameter1, Customization_Parameter2)
        Cerebro_Object.addstrategy(Customize_StrategyObject)

        Im_Data = backtrader.feeds.YahooFinanceData(
            dataname = self.Trading_Object_Name, 
            fromdate=self.Trading_DateTime_Start, 
            todate=self.Trading_DateTime_End)    

        Cerebro_Object.adddata(Im_Data)
        Cerebro_Object.broker.setcash(10000.0)
        Cerebro_Object.run()
        
        self.Customization_Parameter1 = Customization_Parameter1
        self.Customization_Parameter2 = Customization_Parameter2                                            
        self.Trading_Cerebro_Object = Cerebro_Object


    def Function_Plotting_Customize(self) : 

    #    cerebro.plot()
        figure = matplotlib.pyplot.figure(figsize=(24.0, 15.0))
        figure = self.Trading_Cerebro_Object.plot()[0][0]
        figure.savefig('Trading_Plot/Customize/TradingCustomizeStratey_Object[{}]_Position[{}%]_Multiplier[{}].jpg'.format(
            self.Trading_Object_Name,
            self.Trading_Cerebro_Object.broker.getvalue()/100,
            self.Trading_Level_Multiplier))
        matplotlib.pyplot.close(figure)
        matplotlib.pyplot.close("all")
        
        
    def Function_Batch_MultipleParameter(self, Trading_Object_Name_List, Trading_Level_Multiplier_List, Customization_Parameter1_Interval, Customization_Parameter2_Interval):

        try:

            for Specific_Trading_Object_Counter in range(len(Trading_Object_Name_List)):
                for Customization_Parameter1 in range(Customization_Parameter1_Interval[0],Customization_Parameter1_Interval[1]):
                    for Customization_Parameter2 in range(Customization_Parameter2_Interval[0] ,Customization_Parameter2_Interval[1]):

                        self.Trading_Object_Name = Trading_Object_Name_List[Specific_Trading_Object_Counter]
                        self.Trading_Level_Multiplier = Trading_Level_Multiplier_List[Specific_Trading_Object_Counter]
                        self.Function_RunTrading_Customize(Customization_Parameter1 , Customization_Parameter2)
                        self.Parameter_Performance_Batch.append([
                            self.Trading_Object_Name,
                            self.Trading_Cerebro_Object.broker.getvalue()/100,
                            self.Customization_Parameter1,
                            self.Customization_Parameter2])                                            
                        self.Function_Plotting_Customize()
                        
                    self.Function_Plotting_StrategyCombination()

        except:
            
            print(sys.exc_info())
            
            
    def Function_Plotting_StrategyCombination(self) :
        
        X_axsis = numpy.array([])
        Y_axsis = numpy.array([])
        Z_axsis = numpy.array([])
        for data in self.Parameter_Performance_Batch :
            Z_axsis = numpy.append( Z_axsis, data[1])
            Y_axsis = numpy.append( Y_axsis, data[2])
            X_axsis = numpy.append( X_axsis, data[3])
            
        print(X_axsis)        
        print(Y_axsis)
        print(Z_axsis)
        print(self.Parameter_Performance_Batch)        
        ParameterPerformance_Figure = matplotlib.pyplot.figure(figsize=(20,20))
        ax = ParameterPerformance_Figure.add_subplot(111,projection='3d')
        ax.scatter3D(X_axsis, Y_axsis, Z_axsis, c=Z_axsis, s=500, cmap='viridis');
        
        matplotlib.pyplot.show()
        matplotlib.pyplot.close(ParameterPerformance_Figure)
        matplotlib.pyplot.close("all")


In [ ]:
# Customize

I_Am_Trading_Instance = Trading_Picture_Generator("TSM", 1000, datetime(2018,1,1), datetime(2019,11,16))
I_Am_Trading_Instance.Function_RunTrading_Customize("I_AM_Input1","I_AM_Input2")
I_Am_Trading_Instance.Function_Plotting_Customize()

In [ ]:
## Taiwan Market ##
#Batch
I_Am_Trading_Instance.Function_Batch_MultipleParameter(
    Trading_Object_Name_List = ["TSM"], 
    Trading_Level_Multiplier_List = [1000],
    Customization_Parameter1_Interval = [5,15], 
    Customization_Parameter2_Interval = [20,30])